# Tensforflow Basics

In [1]:
import tensorflow as tf

/Users/rthallam/miniconda2/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Tensor
- Data is encapsulated in an object called a tensor - not an integer, string or float
- `hello_constant` is a 0-dimensional string tensor

### Session
TensorFlow’s API is built around the idea of a computational graph, a way of visualizing a mathematical process. The session is in charge of allocating the operations to GPU(s) and/or CPU(s), including remote machines.
<img src="https://d17h27t6h515a5.cloudfront.net/topher/2016/October/580feadb_session/session.png" alt="Session" style="width: 100px;height=50px"/>

In [4]:
# Create TensorFlow object called tensor
hello_constant = tf.constant('Hello World!')
# A is a 0-dimensional int32 tensor
A = tf.constant(1234) 
# B is a 1-dimensional int32 tensor
B = tf.constant([123,456,789]) 
# C is a 2-dimensional int32 tensor
C = tf.constant([ [123,456,789], [222,333,444] ])

with tf.Session() as sess:
    # Run the tf.constant operation in the session
    output = sess.run(hello_constant)
    print(output)
    print(sess.run(A))
    print(sess.run(B))
    print(sess.run(C))
    

b'Hello World!'
1234
[123 456 789]
[[123 456 789]
 [222 333 444]]


### Placeholder

- `tf.placeholder(<datatype>)` returns a tensor that gets its value from data passed to the tf.session.run() function
- Allows to set the input right before the session runs using `feed_dict`

In [8]:
x = tf.placeholder(tf.string)

with tf.Session() as sess:
    output = sess.run(x, feed_dict={x: 'Hello World'})
    print(output)

Hello World


### Tensorflow Math

In [14]:
add = tf.add(5, 2)
sub = tf.subtract(10, 4)
mul = tf.multiply(2, 5)

with tf.Session() as sess:
    # running multiple tensor operations
    output = sess.run([add, sub, mul])
    print(output)

[7, 6, 10]


- Use `tf.cast` for casting data types

In [19]:
with tf.Session() as sess:
    sub = tf.subtract(tf.cast(tf.constant(2.0), tf.int32), tf.constant(1))
    print(sess.run(sub))

1


In [23]:
# Solution is available in the other "solution.py" tab
import tensorflow as tf

# TODO: Convert the following to TensorFlow:
x = tf.constant(10)
y = tf.constant(2)
z = x/y - 1

# TODO: Print z from a session

with tf.Session() as sess:
    # running multiple tensor operations
    output = sess.run(z)
    print(output)

4.0


### Variables

- Tensors created from `tf.placeholder()` and `tf.constant()` cannot be modified. This is where `tf.Variable` class comes in.
    - `tf.Variable` class creates a tensor with an initial value
    - `tf.global_variables_initializer()` function nitialize all TensorFlow variables from the graph

In [24]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

- `tf.truncated_normal()` function generates random numbers from a normal distribution

In [33]:
n_features = 120
n_labels = 5
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))

## Linear Classifier Quiz

![MNIST](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/582cf7a7_mnist-012/mnist-012.png)

Subset of MNIST dataset.

You'll be classifying the handwritten numbers 0, 1, and 2 from the MNIST dataset using TensorFlow. The above is a small sample of the data you'll be training on. Notice how some of the 1s are written with a serif at the top and at different angles. The similarities and differences will play a part in shaping the weights of the model.

![MNIST](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/582ce9ef_weights-0-1-2/weights-0-1-2.png)

Left: Weights for labeling 0. Middle: Weights for labeling 1. Right: Weights for labeling 2.

The images above are trained weights for each label (0, 1, and 2). The weights display the unique properties of each digit they have found. Complete this quiz to train your own weights using the MNIST dataset.

In [34]:
import tensorflow as tf

def get_weights(n_features, n_labels):
    """
    Return TensorFlow weights
    :param n_features: Number of features
    :param n_labels: Number of labels
    :return: TensorFlow weights
    """
    weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))
    return weights


def get_biases(n_labels):
    """
    Return TensorFlow bias
    :param n_labels: Number of labels
    :return: TensorFlow bias
    """
    biases = tf.Variable(tf.zeros(n_labels))
    return biases


def linear(input, w, b):
    """
    Return linear function in TensorFlow
    :param input: TensorFlow input
    :param w: TensorFlow weights
    :param b: TensorFlow biases
    :return: TensorFlow linear function
    """
    return tf.matmul(input, w) + b

In [38]:
# Solution is available in the other "sandbox_solution.py" tab
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

def mnist_features_labels(n_labels):
    """
    Gets the first <n> labels from the MNIST dataset
    :param n_labels: Number of labels to use
    :return: Tuple of feature list and label list
    """
    mnist_features = []
    mnist_labels = []

    mnist = input_data.read_data_sets('~/wrk/datasets/ud730/mnist', one_hot=True)

    # In order to make quizzes run faster, we're only looking at 10000 images
    for mnist_feature, mnist_label in zip(*mnist.train.next_batch(10000)):

        # Add features and labels if it's for the first <n>th labels
        if mnist_label[:n_labels].any():
            mnist_features.append(mnist_feature)
            mnist_labels.append(mnist_label[:n_labels])

    return mnist_features, mnist_labels


# Number of features (28*28 image is 784 features)
n_features = 784
# Number of labels
n_labels = 3

# Features and Labels
features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.float32)

# Weights and Biases
w = get_weights(n_features, n_labels)
b = get_biases(n_labels)

# Linear Function xW + b
logits = linear(features, w, b)

# Training data
train_features, train_labels = mnist_features_labels(n_labels)

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    # Softmax
    prediction = tf.nn.softmax(logits)

    # Cross entropy
    # This quantifies how far off the predictions were.
    # You'll learn more about this in future lessons.
    cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)

    # Training loss
    # You'll learn more about this in future lessons.
    loss = tf.reduce_mean(cross_entropy)

    # Rate at which the weights are changed
    # You'll learn more about this in future lessons.
    learning_rate = 0.08

    # Gradient Descent
    # This is the method used to train the model
    # You'll learn more about this in future lessons.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Run optimizer and get loss
    _, l = session.run(
        [optimizer, loss],
        feed_dict={features: train_features, labels: train_labels})

# Print loss
print('Loss: {}'.format(l))


Extracting ~/wrk/datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting ~/wrk/datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting ~/wrk/datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting ~/wrk/datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz
Loss: 8.412769317626953


### Softmax Function

![softmax](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a3b336_softmax/softmax.png)

In [41]:
import tensorflow as tf

def run():
    output = None
    logit_data = [2.0, 1.0, 0.1]
    logits = tf.placeholder(tf.float32)
    
    # TODO: Calculate the softmax of the logits
    softmax = tf.nn.softmax(logit_data)
    
    with tf.Session() as sess:
        # TODO: Feed in the logit data
        output = sess.run(softmax)

    return output

In [42]:
run()

array([0.6590012 , 0.24243298, 0.09856589], dtype=float32)

### Cross Entropy

![cross entropy](https://d17h27t6h515a5.cloudfront.net/topher/2017/February/589b18f5_cross-entropy-diagram/cross-entropy-diagram.png)

In [44]:
# Solution is available in the other "solution.py" tab
import tensorflow as tf

softmax_data = [0.7, 0.2, 0.1]
one_hot_data = [1.0, 0.0, 0.0]

softmax = tf.placeholder(tf.float32)
one_hot = tf.placeholder(tf.float32)

cross_entropy = -tf.reduce_sum(one_hot_data * tf.log(softmax_data))

# TODO: Print cross entropy from session
with tf.Session() as sess:
    # TODO: Feed in the logit data
    output = sess.run(cross_entropy)
    print(output)

0.35667497


### Mini-Batch

In [47]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Import MNIST data
mnist = input_data.read_data_sets('~/wrk/datasets/ud730/mnist', one_hot=True)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

Extracting ~/wrk/datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting ~/wrk/datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting ~/wrk/datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting ~/wrk/datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz


In [57]:
print(train_features.nbytes)
print(train_labels.nbytes)

172480000
2200000


In [ ]:
# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

In [63]:
from quiz import batches
from pprint import pprint

# 4 Samples of features
example_features = [
    ['F11','F12','F13','F14'],
    ['F21','F22','F23','F24'],
    ['F31','F32','F33','F34'],
    ['F41','F42','F43','F44']]
# 4 Samples of labels
example_labels = [
    ['L11','L12'],
    ['L21','L22'],
    ['L31','L32'],
    ['L41','L42']]

# PPrint prints data structures like 2d arrays, so they are easier to read
pprint(batches(3, example_features, example_labels))


ModuleNotFoundError: No module named 'quiz'

In [62]:
import math
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    mini_batches = []
    size = len(features)
    for p in range(0, size, batch_size):
        b_features = features[p: p + batch_size]
        b_labels = labels[p: p + batch_size]
        mini_batches.append([b_features, b_labels])
    return mini_batches

In [64]:
import math
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches

In [67]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

learning_rate = 0.001
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Import MNIST data
mnist = input_data.read_data_sets('~/wrk/datasets/ud730/mnist', one_hot=True)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# TODO: Set batch size
batch_size = 128
assert batch_size is not None, 'You must set the batch size'

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    # TODO: Train optimizer on all batches
    for batch_features, batch_labels in batches(batch_size, train_features, train_labels):
        sess.run(optimizer, feed_dict={features: batch_features, labels: batch_labels})

    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})

print('Test Accuracy: {}'.format(test_accuracy))

Extracting ~/wrk/datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting ~/wrk/datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting ~/wrk/datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting ~/wrk/datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Test Accuracy: 0.08780000358819962
